# My first ubermag simulation

## 1. Setup

Before defining our system to simulate, and then performing the computation work, we first need to import the required modules. In the case of Ubermag this means importing specific micromagnetic models and other such tools.

The `mm` module contain the core `mm.System` object which we use to specify the physical system; not the mesh. For now, we'll just give it a name.

In [3]:
import micromagneticmodel as mm
import discretisedfield as df

In [5]:
system = mm.System(name='first_ubermag_simulation')

## 2. System Objects

The micromagnetic system will be defined through

1. An energy equation that will be used to define the effective mangetic field $H_{eff}$, and in turn the magnetisation dynamics of the system.
2. A mesh which represents the dimensions (any shapes can be quickly defined with helper functions) of the physical object/structure
3. The initial magnetisation state of the structure

### 2.1. Energy equation

In [6]:
A = 1e-12  # exchange energy constant (J/m)

# While Ubermag denotes this as a tuple, Mumax3 would have it as a vector ( H_ext = vector(x,y,z)
H = (0, 0, 5e6)  # external magnetic field (A/m). 

# See the documentation for a full list of methods and arguments.
system.energy = mm.Exchange(A=A) + mm.Demag() + mm.Zeeman(H=H)

### 2.2. Mesh

Here we'll be simulating a cube with edges of lengths $50~nm$. We'll then discretise it into $8$ cells along each cardinal axis; meaning 512 cells in total). Once should choose a number of cells (along each axis) following these guidelines (from best to worst). The number of cells:

- is a power of 2.
- has only a few factors, and each is a small valued prime number (*i.e.* 14 has factors of 2 and 7 which is reasonable)
- is itself a prime number that is small (*i.e.* 31 is better than 37)
- are not too large

The final point is important as excessive discretisation of a mesh can lead to the micromagnetic solvers producing nonsense, just as too few cells can lead to a macroscopic representation of a microscopic system, which also leads to errors.

In [9]:
L = 50e-9  # cubic sample edge length (m)
num_cells = 8  # number of cells along each axis

# Ubermag knows if three points are provided where p1[0]=p1[1]=p1[2] and p2[0]=p2[1]=p2[2] then the shape is a cube
region = df.Region(p1=(0,0,0), p2=(L,L,L))  # p1 and p2 are the parameters used to define a line. 

mesh = df.Mesh(region=region, n=(num_cells,num_cells,num_cells))

### 2.3. Initial magenetistaion

Here we'll keep things simply. We'll initialise the system in a positive $\hat{z}$ direction, while the equilibrium state due to $H_0$ is along the $\hat{z}$ axis.

In [10]:
Ms = 8e6  # saturisation magnetisation (A/m)
m_init = (0, 0, 1)  # initial reduced magnetisation
num_dim = 3  # number of dimensions in this mesh

system.m = df.Field(mesh, nvdim=num_dim, value=m_init, norm=Ms)

## 3. Inspecting this new system object

We now have our system object `mm.System`! One of the fantastic aspects of Ubermag, over using the micromagnetic calculators directly, is that we are able to have symbolic interactions with our equations. This is useful for gaining intuition, and looking for mistakes in the implementations of our various equations.

### 3.1. Energy equation

This, for example, is the energy equation we just defined:

In [11]:
system.energy

### 3.2. Mesh

It can be quite hard to visualise complex system's meshes (wireframes and more are great ways), so for now we'll just look at a simple `matplotlib` rendering (similar to how Qiskit abstracts Qubits). On the below figure the blue cube is the full mesh, and the orange cube is our cell.

In [15]:
system.m.mesh.mpl()
# system.m.mesh.region.mpl()  # use this if you don't want to see the cell

### 3.3. Magnetisation

The magnetisation is best seen in these simple cases as a 2D slice of the magnetisation. We know that our earlier setup worked through the observations:

- The $\hat{x}$ and $\hat{y}$ directions are aligned along the $\hat{z}$ direction, and these magnetisation components are both equal and small in magnitude (compared to the $\hat{h}$ direction).
- The $\hat{z}$ direction is entirely saturated.

Not that we don't have arrows on the $x-y$ plane to show if the magnetisation component is in or out of the plane. Instead on all figures we look to the colourbar on the righthand side for this information. We can therefore note that the positive values of the z-component in the $x-y$ plane show we are aligned along $+\hat{z}$.

In [32]:
for select_axis in ['x', 'y', 'z']:
    system.m.sel(select_axis).mpl()

## 4. Driving the system

Now that we're all setup, it's time to consider how we want to drive the system *i.e.* introduce spatially- and temporally-dependent parameters. We also need to choose the micromagnetic calculator (`mumax3c`, `oommfc`, *etc*) we want to implement. 

### 4.1. Select a calculator

Here I'm going to use `oommfc` because it is more completely supported by Ubermag.

In [34]:
import oommfc as mc  # the name comes from Object Orientationed MicroMagnetic Framework (OOMMF) Calculator (c)

We will first minimise the system's energy through a relaxation process controlled by the Minimisation Driver `MinDriver`. For the first time you see a popup stating *Running OOMMF*; check the terminal where Jupyter was called to see the steps that OOMMF performed!

In [35]:
md = mc.MinDriver()
md.drive(system)

The system is now relaxed, and we have updated the magnetisation. This is a destructive process because we are overwriting the values held within `system.m`. If you want to reuse a given system then be sure to use a unique declaration like `system_base`. Also note that each `Driver` will be applied sequentially so be sure that you call each `Driver` in the correct order!

Let's take a look at our relaxed system, and demonstrate a few computations.

In [80]:
# Helper function to format numpy arrays
def formatted_array(input_array):
    return ", ".join(f"{x:.3e}" for x in input_array)

print(f'Mean: {formatted_array(system.m.mean())}')

mean_axes = system.m.orientation.mean()
print(f'Mean (orientation): {formatted_array(mean_axes)}')

# Showing how your operations aren't mutating `system`
print(f'Mean (sum): {sum([value for value in mean_axes]):.3f}')

In [81]:
# Just another way to loop over consecutive characters
for select_axis in range(ord('x'), ord('z')+1):
    resample_num_cells = num_cells  # Try changing to see what happens!
    system.m.sel(chr(select_axis)).resample((resample_num_cells, resample_num_cells)).mpl()

## 5. Final result

The magnetisation is aligned along the $\hat{x}$ axis which is what we expected for the geometry we selected. Success!